In [1]:
import jax.numpy as jnp
from ddfa_node import sample_floquet_multipliers, backtrace_multipliers
import matplotlib.pyplot as plt

# load reload extension:
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'ddfa_node'

In [ ]:
generated_data = jnp.load("outputs/gen_human_data.npy")
print(generated_data.shape)

In [ ]:
plt.plot(generated_data[0, :, 0])


In [ ]:
allEigenvals, allEigenvecs, allRs, allPhis, Ns = sample_floquet_multipliers(
    generated_data,
    nSegments=20, 
    nCovReps=1, 
    phaser_feats=None, 
    splits=range(2, 5), 
    nReplicates=300, 
    usePCA=False, 
    height=0.85, 
    distance=100, 
    vecs=False)

In [ ]:
backtrace_multipliers(splits, eigVals, Ns, subject=0, nPoints=5, phase=50, eig=0, plot=True, plot_title=None, ax=None)